# Annotate new data to improve NLP models using Rubrix and biome.text

## Introduction

Hey there! In this guide, we will show you how to use Rubrix to annotate new data, and use this new data to improve existing Deep Learning models. Our use case will be Automatic Misogyny Detection (AMI): Deep Learning models able to detect the underlying misogyny on a given text. Ground-breaking work is being made every year on this subject, with shared tasks and new models that push the performance of these models closer and closer to be implemented in apps, social networks and other digital environments. 

To train these NLP models we are going to use [biome.text](https://github.com/recognai/biome-text), an open-source library to train models with a simple workflow. Rubrix is compatible with almost any library or service, so we will work back and forth with both of them. 

The data used to feed the models and make the annotations comes from the [IberEval 2018](https://sites.google.com/view/ibereval-2…) shared task. We are also making the specific datasets used in each step of this guide available, so it can be reproduced in the best way possible.

## Dependencies

If you want to reproduce this code, make sure that all the libraries needed to run this guide are installed and imported.

In [ ]:
%pip install -U git+https://github.com/recognai/biome-text
%pip install rubrix
%pip install pandas
exit(0)  # Force restart of the runtime

In [ ]:
import os

os.environ['WANDB_API_KEY'] = '7bd265df21100baa9767bb9f69108bc417db4b4a'

In [1]:
from biome.text import *
import pandas as pd
import rubrix as rb

import wandb #TODO: erase

## Training the first model

Hey there! In this guide, we will show you how to use Rubrix to annotate new data, and use this new data to improve existing Deep Learning models. Our use case will be Automatic Misogyny Detection (AMI): Deep Learning models able to detect the underlying misogyny on a given text. Ground-breaking work is being made every year on this subject, with shared-tasks and new models that push the performance of this model closer and closer to be implemented in apps, social networks and other digital environments. 

To train these NLP models we are going to use [biome.text](https://github.com/recognai/biome-text), an open-source library to train models with a simple workflow. Rubrix is compatible with almost any library or service, so we will work back and forth with both of them. 

The data used to feed the models and make the annotations comes from the [IberEval 2018](https://sites.google.com/view/ibereval-2…) shared-task. We are also making the specific datasets used in each step of this guide available, so it can be reproduced in the best way possible.

## HPO (wont be included)

In [ ]:
training_ds = Dataset.from_pandas(training_df)
validation_ds = Dataset.from_pandas(validation_df)
test_ds = Dataset.from_pandas(test_df)
training_full_ds = Dataset.from_pandas(training_full_df)

In [ ]:
pipeline_dict = {
    "name": "rubrix_guide",
    "features": {
        "transformers": {
            "model_name": "dccuchile/bert-base-spanish-wwm-cased",
            "trainable": True,
            "max_length": 280,  #twitter characters cap
        }
    },
    "head": {
        "type": "TextClassification",
        "multilabel": True,
        "labels": [
            'sexual_harassment',
             'dominance',
             'discredit',
             'stereotype',
             'derailing',
             'passive',
             'active',
             '0'
        ],
        "pooler": {
            "type": tune.choice(["gru", "lstm"]),
            "num_layers": 1,
            "hidden_size": tune.choice([32,64,128,256]),
            "bidirectional": tune.choice([True, False]),
        },
    },
}

In [ ]:
batch_size = 16
trainer_dict = {
    "optimizer": {
        "type": "adamw",
        "lr": tune.loguniform(1e-5, 1e-4),
        "weight_decay": tune.loguniform(2e-3, 6e-2 )
    },
    "learning_rate_scheduler": {
        "type": "linear_with_warmup",
        "num_epochs": 10,
        "num_steps_per_epoch": int(math.floor(len(training_ds)/batch_size)),
        "warmup_steps": 100,
    },
    "batch_size": batch_size,
    "num_epochs": 10,
    
}

In [ ]:
search_alg = HyperOptSearch(metric="validation_macro/fscore", mode="max")

In [ ]:
tune_exp = TuneExperiment(
    pipeline_config=pipeline_dict, 
    trainer_config=trainer_dict,
    train_dataset=training_ds,
    valid_dataset=validation_ds,
    name="rubrix_guide",
    # parameters for tune.run
    num_samples=100,
    local_dir="tune_runs",
    resources_per_trial={"cpu": 2, "gpu": 1}
)

In [ ]:
analysis_frozen = tune.run(
    tune_exp,
    scheduler=tune.schedulers.ASHAScheduler(), 
    config=tune_exp.config,
    metric="validation_macro/fscore",
    search_alg=search_alg,
    mode="max",
    progress_reporter=tune.JupyterNotebookReporter(overwrite=True),
)